### AWS Simple Storage Service (S3)
##### Disable Server Access Logging

In [ ]:
import boto3
import botoplus.organization as _org

accounts = _org.accounts()
selected = _org.selected()

home = 'us-east-2' # Define Management Region

for account in accounts:

    if account['awsalias'] == 'Log archive': # Exclude Management Account

        print(str(account['awsaccount'])+' '+account['awsalias'])
        print(' - '+home)

        sts = selected.client('sts')

        assumed = sts.assume_role(
            RoleArn = 'arn:aws:iam::'+account['awsaccount']+':role/AWSControlTowerExecution',
            RoleSessionName = 'ControlTowerBreakGlass'
        )
        session = boto3.Session(
            aws_access_key_id = assumed['Credentials']['AccessKeyId'],
            aws_secret_access_key = assumed['Credentials']['SecretAccessKey'],
            aws_session_token = assumed['Credentials']['SessionToken']
        )

        s3 = session.client('s3', region_name=home)

        try:
            s3.put_bucket_logging(
                Bucket = 'aws-controltower-logs-'+str(account['awsaccount'])+'-'+home,
                BucketLoggingStatus = {}
            )
            print('   Server Access Logging Disabled')
        except Exception as e:
            print('   Error: '+str(e))
            pass